In [1]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib inline
#inline

# Importamos las librerías necesarias
import numpy as np
import time
import cv2 as cv
import matplotlib.pyplot as plt
import os

In [3]:
# Defino una función para comparar histogramas
def hist_comp(img1, img2):

    bins1 = cv.calcHist([img1],[0],None,[256],[0,256])
    bins2 = cv.calcHist([img2],[0],None,[256],[0,256])

    cv.normalize(bins1, bins1, alpha=0, beta=1, norm_type=cv.NORM_MINMAX)
    cv.normalize(bins2, bins2, alpha=0, beta=1, norm_type=cv.NORM_MINMAX)
    #histfloat1= np.float32(bins1)
    #histfloat2= np.float32(bins2)
    metric_val = cv.compareHist(bins1, bins2, cv.HISTCMP_CORREL)
    return metric_val

## Cargamos el clasificador pre-entrenado

In [4]:
#--------------------------------------------------------------------------------
face_cascade = cv.CascadeClassifier(r'C:\Users\Aulamultimedia\haarcascade_frontalface_alt.xml')
eye_cascade = cv.CascadeClassifier(r'C:\Users\Aulamultimedia\haarcascade_eye.xml')

## Hacemos el seguimiento de frames desde la webcam de la pc

In [16]:
# Condición de terminación: 10 iteraciones o moverse al menos 1pt
#----------------------------------------------------------------
term_crit = ( cv.TERM_CRITERIA_COUNT | cv.TERM_CRITERIA_EPS, 10, 1 )
cap = cv.VideoCapture(0)
# Initiate ORB detector
sift = cv.SIFT_create()
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv.VideoWriter('pestaneo.avi',cv.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
# inicializamos parametros y contadores
param = 0
metricas = 0
metricasP = 0
primera_lectura = True
contador = 0
contador2 = 0
n = 5
while(1):

    ret ,frame = cap.read()
    contador = contador + 1
    if contador == n: contador = 0
    if contador == 4: contador2 = contador2 + 1

    if ret == True:
        key = False
        

        if key == True:
            break
        else:
            originalImage = frame
            # Imagen a escala de grises
            grayImage = cv.cvtColor(originalImage, cv.COLOR_BGR2GRAY)
            grayImage = cv.equalizeHist(grayImage)

            # Llamamos al clasificador de Haar (AdaBoost)

            faces = face_cascade.detectMultiScale(grayImage, 1.1, 1)

            # Recorro las caras encontradas
            #------------------------------
            for (x,y,w,h) in faces:
                cv.putText(originalImage, 'Presione \"q\" para salir',(80,450), cv.FONT_HERSHEY_PLAIN, 1.5,(0,255,255),2)
                # Le dibujamos un rectángulo rosa
                cv.rectangle(originalImage,(x,y),(x+w,y+h),(255,0,255),2)
                # Definimos las ROIs en la imagen gris y color
                roi_gray = grayImage[y:y+h, x:x+w] 
                roi_color = originalImage[y:y+h, x:x+w] 
                # Para cada rostro hallado le buscamos los ojos
                eyes = eye_cascade.detectMultiScale(roi_gray, 1.2, 8)
                # En los ojos hallados les dibujamos rectángulos azules
                for (ex,ey,ew,eh) in eyes:
                    cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,0,0),2)
                cv.putText(originalImage, str(len(eyes)),(10,30), cv.FONT_HERSHEY_PLAIN, 1,(0,0,0),2)
                # Detecto si el programa está viendo los ojos
                if (primera_lectura) and (len(eyes)==2):
                    cv.putText(originalImage, 'Ojos detectados, \"s\" para empezar',(20,80), cv.FONT_HERSHEY_PLAIN, 2,(255,255,0),2)
                elif primera_lectura == False:
                    # ubico el ojo en la posición relativa comparando con el primer frame
                    ojo1 = roi_gray[ojo_rel1_y:(ojo_rel1_y+ojo_rel1_h), ojo_rel1_x:(ojo_rel1_x+ojo_rel1_w)]
                    ojo2 = roi_gray[ojo_rel2_y:(ojo_rel2_y+ojo_rel2_h), ojo_rel2_x:(ojo_rel2_x+ojo_rel2_w)]
                    
                    # Calculo las difrencias entre histogramas de la imagen inicial con la actual
                    metrica1 = hist_comp(ojo1, ojo1_f)
                    metrica2 = hist_comp(ojo2, ojo2_f)

                    # promedio móvil de 5 frames
                    if contador == 0: metricas = 0 
                    metricas = metricas + metrica1 + metrica2
                    if contador == n-1: metricasP = metricas/n
                    cv.putText(originalImage, 'param.: ' + str(metricasP),(10,140), cv.FONT_HERSHEY_PLAIN, 2,(0,255,0),2)
                    
                    # Se muestra el pestaneo en pantalla, evitando los primeros 20 frames    
                    if metricasP <= 1 and contador2 >= 4:
                        cv.putText(originalImage, 'PESTANEO DETECTADO',(10,300), cv.FONT_HERSHEY_PLAIN, 3,(0,255,0),2)

            cv.imshow('Deteccion de pestaneos!',originalImage)
            w = cv.waitKey(1)
            if(w==ord('q')):
                break
            elif(w==ord('s') and primera_lectura):
                #This will start the detection
                if(len(eyes)==2):
                    primera_lectura = False

                    ojo_rel1_x = eyes[0][0]
                    ojo_rel1_y = eyes[0][1]
                    ojo_rel1_w = eyes[0][2]
                    ojo_rel1_h = eyes[0][3]
                    ojo1_f = roi_gray[ey:ey+eh, ex:ex+ew]
                    centro_ojo1_f = hist_center(ojo1_f)

                    ojo_rel2_x = eyes[1][0]
                    ojo_rel2_y = eyes[1][1]
                    ojo_rel2_w = eyes[1][2]
                    ojo_rel2_h = eyes[1][3] 
                    ojo2_f = roi_gray[ey:ey+eh, ex:ex+ew]
                    centro_ojo2_f = hist_center(ojo1_f)
                
            out.write(frame)

    else:
        break


cap.release()
out.release()
cv.destroyAllWindows()